<a href="https://colab.research.google.com/github/babi2707/Trabalho-PAI---Reconhecimento-Papanicolau/blob/etapa2/algoritmo/backend/Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# Célula 1:
import zipfile

caminho_pasta_compactada = '/content/sub_images.zip'
caminho_destino = '/content/sub_images'

with zipfile.ZipFile(caminho_pasta_compactada, 'r') as zip_ref:
    zip_ref.extractall(caminho_destino)

In [51]:
# Célula 2:
import os

base_dir = '/content/sub_images/sub_images'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

classes = ['ASC-H', 'ASC-US', 'HSIL', 'LSIL', 'Negative for intraepithelial lesion', 'SCC']
for class_name in classes:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

In [52]:
# Célula 3:
from sklearn.model_selection import train_test_split
import os
import numpy as np

# Diretório onde estão os dados
data_dir = '/content/sub_images/sub_images'

# Lista para armazenar os nomes dos arquivos e suas respectivas classes
file_names = []
labels = []

# Preencher as listas com os nomes dos arquivos e suas classes correspondentes
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        file_names.append(file_path)
        labels.append(class_name)

# Dividir os dados em treino e teste mantendo a proporção de 4:1
train_files = []
test_files = []
train_labels = []
test_labels = []

# Para cada classe, dividir os arquivos em treino e teste mantendo a proporção de 4:1
for class_name in set(labels):
    class_files = [file for file, label in zip(file_names, labels) if label == class_name]
    class_labels = [label for label in labels if label == class_name]

    # Dividir os arquivos e rótulos em treino e teste
    class_train_files, class_test_files, class_train_labels, class_test_labels = train_test_split(
        class_files, class_labels, test_size=0.2, random_state=42)

    # Adicionar os arquivos e rótulos aos conjuntos de treino e teste
    train_files.extend(class_train_files)
    test_files.extend(class_test_files)
    train_labels.extend(class_train_labels)
    test_labels.extend(class_test_labels)

# Verificar o balanceamento das classes nos conjuntos de treino e teste
unique, counts = np.unique(train_labels, return_counts=True)
print(f'Balanceamento das classes no conjunto de treino:')
for class_name, count in zip(unique, counts):
    print(f'Classe {class_name}: {count} amostras')

unique, counts = np.unique(test_labels, return_counts=True)
print(f'Balanceamento das classes no conjunto de teste:')
for class_name, count in zip(unique, counts):
    print(f'Classe {class_name}: {count} amostras')

Balanceamento das classes no conjunto de treino:
Classe ASC-H: 67 amostras
Classe ASC-US: 312 amostras
Classe HSIL: 229 amostras
Classe LSIL: 394 amostras
Classe Negative for intraepithelial lesion: 800 amostras
Classe SCC: 65 amostras
Balanceamento das classes no conjunto de teste:
Classe ASC-H: 17 amostras
Classe ASC-US: 79 amostras
Classe HSIL: 58 amostras
Classe LSIL: 99 amostras
Classe Negative for intraepithelial lesion: 200 amostras
Classe SCC: 17 amostras


In [53]:
# Célula 4:
import shutil
import random
import os

def split_data(source_dir, train_dir, test_dir):
    for class_name in classes:
        class_dir = os.path.join(source_dir, class_name)
        train_class_dir = os.path.join(train_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)

        for image in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image)
            if image_path in train_files:
                dest_path = os.path.join(train_class_dir, image)
                if not os.path.exists(dest_path):
                    shutil.copy2(image_path, dest_path)
                else:
                    # Renomear o arquivo se já existir
                    base, ext = os.path.splitext(image)
                    new_dest_path = os.path.join(train_class_dir, f"{base}_copy{ext}")
                    shutil.copy2(image_path, new_dest_path)
            elif image_path in test_files:
                dest_path = os.path.join(test_class_dir, image)
                if not os.path.exists(dest_path):
                    shutil.copy2(image_path, dest_path)
                else:
                    # Renomear o arquivo se já existir
                    base, ext = os.path.splitext(image)
                    new_dest_path = os.path.join(test_class_dir, f"{base}_copy{ext}")
                    shutil.copy2(image_path, new_dest_path)

split_data(base_dir, train_dir, test_dir)


In [54]:
# Célula 5:
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler

def extract_histogram_features(image_path, h_bins=16, s_bins=8):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_image], [0, 1], None, [h_bins, s_bins], [0, 180, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

train_features = []
train_labels_hist = []
test_features = []
test_labels_hist = []

for file_path, label in zip(train_files, train_labels):
    features = extract_histogram_features(file_path)
    train_features.append(features)
    train_labels_hist.append(label)

for file_path, label in zip(test_files, test_labels):
    features = extract_histogram_features(file_path)
    test_features.append(features)
    test_labels_hist.append(label)

train_features = np.array(train_features)
test_features = np.array(test_features)

# Codificar as classes
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels_hist)
test_labels_encoded = label_encoder.transform(test_labels_hist)

# Padronizar as características
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

In [55]:
# Célula 6:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Classificador binário (classe negativa X demais)
binary_labels_train = np.where(train_labels_encoded == 4, 0, 1)  # 'Negative for intraepithelial lesion' = 4
binary_labels_test = np.where(test_labels_encoded == 4, 0, 1)

svm_binary = SVC(kernel='linear')
svm_binary.fit(train_features_scaled, binary_labels_train)
binary_predictions = svm_binary.predict(test_features_scaled)

# Classificador multiclasse (6 classes)
svm_multiclass = SVC(kernel='linear')
svm_multiclass.fit(train_features_scaled, train_labels_encoded)
multiclass_predictions = svm_multiclass.predict(test_features_scaled)

# Avaliação do classificador binário
binary_accuracy = accuracy_score(binary_labels_test, binary_predictions)
binary_conf_matrix = confusion_matrix(binary_labels_test, binary_predictions)
print('Binary Classifier Accuracy:', binary_accuracy)
print('Binary Classifier Confusion Matrix:')
print(binary_conf_matrix)
print('Binary Classifier Classification Report:')
print(classification_report(binary_labels_test, binary_predictions, target_names=['Negative', 'Others']))

# Avaliação do classificador multiclasse
multiclass_accuracy = accuracy_score(test_labels_encoded, multiclass_predictions)
multiclass_conf_matrix = confusion_matrix(test_labels_encoded, multiclass_predictions)
print('Multiclass Classifier Accuracy:', multiclass_accuracy)
print('Multiclass Classifier Confusion Matrix:')
print(multiclass_conf_matrix)
print('Multiclass Classifier Classification Report:')
print(classification_report(test_labels_encoded, multiclass_predictions, target_names=label_encoder.classes_))

Binary Classifier Accuracy: 0.7148936170212766
Binary Classifier Confusion Matrix:
[[154  46]
 [ 88 182]]
Binary Classifier Classification Report:
              precision    recall  f1-score   support

    Negative       0.64      0.77      0.70       200
      Others       0.80      0.67      0.73       270

    accuracy                           0.71       470
   macro avg       0.72      0.72      0.71       470
weighted avg       0.73      0.71      0.72       470

Multiclass Classifier Accuracy: 0.5468085106382978
Multiclass Classifier Confusion Matrix:
[[  6   4   4   1   2   0]
 [  7  18   6  10  38   0]
 [  0   6  33   7   6   6]
 [  2   6   9  23  56   3]
 [  0  12   3  10 175   0]
 [  0   5   6   2   2   2]]
Multiclass Classifier Classification Report:
                                     precision    recall  f1-score   support

                              ASC-H       0.40      0.35      0.38        17
                             ASC-US       0.35      0.23      0.28      

In [57]:
#Rede Neural (1.000 imagens de cada classe):

# Found 2037 images belonging to 6 classes.
# Found 805 images belonging to 6 classes.
# Found 797 images belonging to 6 classes.
# Epoch 1/30
# 64/64 [==============================] - 16s 184ms/step - loss: 13.5700 - accuracy: 0.2641 - val_loss: 12.0976 - val_accuracy: 0.0509
# Epoch 2/30
# 64/64 [==============================] - 9s 148ms/step - loss: 10.2440 - accuracy: 0.3211 - val_loss: 9.1001 - val_accuracy: 0.1938
# Epoch 3/30
# 64/64 [==============================] - 9s 137ms/step - loss: 8.0299 - accuracy: 0.3397 - val_loss: 7.2859 - val_accuracy: 0.2075
# Epoch 4/30
# 64/64 [==============================] - 9s 133ms/step - loss: 6.5430 - accuracy: 0.3579 - val_loss: 6.0852 - val_accuracy: 0.3143
# Epoch 5/30
# 64/64 [==============================] - 9s 148ms/step - loss: 5.5302 - accuracy: 0.3760 - val_loss: 5.9909 - val_accuracy: 0.3143
# Epoch 6/30
# 64/64 [==============================] - 9s 146ms/step - loss: 4.8359 - accuracy: 0.3864 - val_loss: 5.0607 - val_accuracy: 0.3143
# Epoch 7/30
# 64/64 [==============================] - 10s 156ms/step - loss: 4.3658 - accuracy: 0.3991 - val_loss: 7.0809 - val_accuracy: 0.0447
# Epoch 8/30
# 64/64 [==============================] - 9s 145ms/step - loss: 4.0533 - accuracy: 0.3962 - val_loss: 5.0719 - val_accuracy: 0.3143
# Epoch 9/30
# 64/64 [==============================] - 10s 149ms/step - loss: 3.7930 - accuracy: 0.4089 - val_loss: 4.1247 - val_accuracy: 0.2509
# Epoch 10/30
# 64/64 [==============================] - 10s 153ms/step - loss: 3.5963 - accuracy: 0.4134 - val_loss: 5.6467 - val_accuracy: 0.3143
# Epoch 11/30
# 64/64 [==============================] - 10s 161ms/step - loss: 3.5038 - accuracy: 0.4006 - val_loss: 5.5816 - val_accuracy: 0.3155
# Epoch 12/30
# 64/64 [==============================] - 8s 133ms/step - loss: 3.3871 - accuracy: 0.4035 - val_loss: 4.5442 - val_accuracy: 0.2012
# Epoch 13/30
# 64/64 [==============================] - 10s 150ms/step - loss: 3.2483 - accuracy: 0.4055 - val_loss: 6.3419 - val_accuracy: 0.3193
# Epoch 14/30
# 64/64 [==============================] - 10s 150ms/step - loss: 3.1760 - accuracy: 0.4089 - val_loss: 10.3866 - val_accuracy: 0.3143
# Epoch 15/30
# 64/64 [==============================] - 9s 136ms/step - loss: 3.0852 - accuracy: 0.4129 - val_loss: 5.1842 - val_accuracy: 0.3143
# Epoch 16/30
# 64/64 [==============================] - 9s 137ms/step - loss: 3.0400 - accuracy: 0.4099 - val_loss: 3.9754 - val_accuracy: 0.2534
# Epoch 17/30
# 64/64 [==============================] - 10s 149ms/step - loss: 2.9687 - accuracy: 0.4148 - val_loss: 12.1024 - val_accuracy: 0.3143
# Epoch 18/30
# 64/64 [==============================] - 9s 146ms/step - loss: 2.9050 - accuracy: 0.4163 - val_loss: 5.5593 - val_accuracy: 0.0522
# Epoch 19/30
# 64/64 [==============================] - 9s 137ms/step - loss: 2.8096 - accuracy: 0.4119 - val_loss: 5.9457 - val_accuracy: 0.0559
# Epoch 20/30
# 64/64 [==============================] - 9s 133ms/step - loss: 2.7621 - accuracy: 0.4129 - val_loss: 5.1548 - val_accuracy: 0.2311
# Epoch 21/30
# 64/64 [==============================] - 9s 142ms/step - loss: 2.7199 - accuracy: 0.4246 - val_loss: 5.6391 - val_accuracy: 0.2161
# Epoch 22/30
# 64/64 [==============================] - 9s 148ms/step - loss: 2.6605 - accuracy: 0.4168 - val_loss: 5.0455 - val_accuracy: 0.3143
# Epoch 23/30
# 64/64 [==============================] - 10s 150ms/step - loss: 2.6190 - accuracy: 0.4207 - val_loss: 4.2873 - val_accuracy: 0.3180
# Epoch 24/30
# 64/64 [==============================] - 8s 130ms/step - loss: 2.5822 - accuracy: 0.4246 - val_loss: 7.6085 - val_accuracy: 0.3143
# Epoch 25/30
# 64/64 [==============================] - 9s 148ms/step - loss: 2.5455 - accuracy: 0.4227 - val_loss: 3.5279 - val_accuracy: 0.2224
# Epoch 26/30
# 64/64 [==============================] - 10s 149ms/step - loss: 2.4830 - accuracy: 0.4408 - val_loss: 6.4590 - val_accuracy: 0.0460
# Epoch 27/30
# 64/64 [==============================] - 9s 136ms/step - loss: 2.4667 - accuracy: 0.4286 - val_loss: 5.8520 - val_accuracy: 0.1267
# Epoch 28/30
# 64/64 [==============================] - 9s 142ms/step - loss: 2.4167 - accuracy: 0.4345 - val_loss: 7.1344 - val_accuracy: 0.3143
# Epoch 29/30
# 64/64 [==============================] - 10s 154ms/step - loss: 2.3761 - accuracy: 0.4256 - val_loss: 5.6150 - val_accuracy: 0.1180
# Epoch 30/30
# 64/64 [==============================] - 11s 166ms/step - loss: 2.3646 - accuracy: 0.4340 - val_loss: 3.6267 - val_accuracy: 0.2298
# Epoch 1/20
# 64/64 [==============================] - 45s 206ms/step - loss: 2.9549 - accuracy: 0.3584 - val_loss: 2.8622 - val_accuracy: 0.3056
# Epoch 2/20
# 64/64 [==============================] - 11s 171ms/step - loss: 2.6053 - accuracy: 0.4423 - val_loss: 3.1694 - val_accuracy: 0.3143
# Epoch 3/20
# 64/64 [==============================] - 12s 184ms/step - loss: 2.5010 - accuracy: 0.4458 - val_loss: 4.9082 - val_accuracy: 0.3056
# Epoch 4/20
# 64/64 [==============================] - 10s 162ms/step - loss: 2.3888 - accuracy: 0.4826 - val_loss: 5.1419 - val_accuracy: 0.3093
# Epoch 5/20
# 64/64 [==============================] - 11s 164ms/step - loss: 2.3569 - accuracy: 0.4752 - val_loss: 6.0688 - val_accuracy: 0.3143
# Epoch 6/20
# 64/64 [==============================] - 11s 176ms/step - loss: 2.3143 - accuracy: 0.4909 - val_loss: 4.9855 - val_accuracy: 0.3193
# Epoch 7/20
# 64/64 [==============================] - 11s 177ms/step - loss: 2.2714 - accuracy: 0.5169 - val_loss: 3.4353 - val_accuracy: 0.2124
# Epoch 8/20
# 64/64 [==============================] - 10s 159ms/step - loss: 2.2721 - accuracy: 0.5115 - val_loss: 2.7955 - val_accuracy: 0.2894
# Epoch 9/20
# 64/64 [==============================] - 11s 173ms/step - loss: 2.2038 - accuracy: 0.5204 - val_loss: 2.9421 - val_accuracy: 0.2770
# Epoch 10/20
# 64/64 [==============================] - 11s 175ms/step - loss: 2.2160 - accuracy: 0.5307 - val_loss: 3.0551 - val_accuracy: 0.2596
# Epoch 11/20
# 64/64 [==============================] - 11s 164ms/step - loss: 2.1450 - accuracy: 0.5479 - val_loss: 3.1095 - val_accuracy: 0.2658
# Epoch 12/20
# 64/64 [==============================] - 10s 156ms/step - loss: 2.1215 - accuracy: 0.5503 - val_loss: 3.1971 - val_accuracy: 0.2683
# Epoch 13/20
# 64/64 [==============================] - 11s 175ms/step - loss: 2.0969 - accuracy: 0.5434 - val_loss: 3.0300 - val_accuracy: 0.2708
# Epoch 14/20
# 64/64 [==============================] - 12s 193ms/step - loss: 2.1205 - accuracy: 0.5557 - val_loss: 2.7476 - val_accuracy: 0.3205
# Epoch 15/20
# 64/64 [==============================] - 11s 175ms/step - loss: 2.0457 - accuracy: 0.5660 - val_loss: 2.4871 - val_accuracy: 0.3739
# Epoch 16/20
# 64/64 [==============================] - 11s 170ms/step - loss: 2.0847 - accuracy: 0.5596 - val_loss: 2.2797 - val_accuracy: 0.4460
# Epoch 17/20
# 64/64 [==============================] - 11s 178ms/step - loss: 2.0516 - accuracy: 0.5680 - val_loss: 2.0958 - val_accuracy: 0.5056
# Epoch 18/20
# 64/64 [==============================] - 12s 186ms/step - loss: 2.0212 - accuracy: 0.5812 - val_loss: 1.9185 - val_accuracy: 0.5652
# Epoch 19/20
# 64/64 [==============================] - 11s 173ms/step - loss: 2.0133 - accuracy: 0.5665 - val_loss: 1.9558 - val_accuracy: 0.5801
# Epoch 20/20
# 64/64 [==============================] - 11s 173ms/step - loss: 1.9954 - accuracy: 0.5650 - val_loss: 1.9267 - val_accuracy: 0.5975
# 25/25 [==============================] - 2s 78ms/step - loss: 1.9346 - accuracy: 0.6123
# Test accuracy: 0.6122961044311523


# Etapa 2 - Número 1 (1.000 imagens de cada classe):
# Found 3904 images belonging to 6 classes.
# Found 805 images belonging to 6 classes.
# Found 1268 images belonging to 6 classes.
# Epoch 1/30
# 122/122 [==============================] - 21s 144ms/step - loss: 12.0839 - accuracy: 0.2941 - val_loss: 9.3830 - val_accuracy: 0.1714
# Epoch 2/30
# 122/122 [==============================] - 17s 136ms/step - loss: 7.5149 - accuracy: 0.3571 - val_loss: 6.1398 - val_accuracy: 0.3143
# Epoch 3/30
# 122/122 [==============================] - 17s 140ms/step - loss: 5.3520 - accuracy: 0.4006 - val_loss: 5.0149 - val_accuracy: 0.2199
# Epoch 4/30
# 122/122 [==============================] - 17s 135ms/step - loss: 4.3241 - accuracy: 0.4050 - val_loss: 4.4759 - val_accuracy: 0.3143
# Epoch 5/30
# 122/122 [==============================] - 17s 135ms/step - loss: 3.8401 - accuracy: 0.4101 - val_loss: 7.2752 - val_accuracy: 0.3143
# Epoch 6/30
# 122/122 [==============================] - 16s 135ms/step - loss: 3.5082 - accuracy: 0.4209 - val_loss: 6.3362 - val_accuracy: 0.1093
# Epoch 7/30
# 122/122 [==============================] - 17s 139ms/step - loss: 3.2837 - accuracy: 0.4280 - val_loss: 5.8864 - val_accuracy: 0.3950
# Epoch 8/30
# 122/122 [==============================] - 17s 140ms/step - loss: 3.1402 - accuracy: 0.4232 - val_loss: 3.4573 - val_accuracy: 0.3950
# Epoch 9/30
# 122/122 [==============================] - 16s 132ms/step - loss: 2.9992 - accuracy: 0.4273 - val_loss: 6.4796 - val_accuracy: 0.3193
# Epoch 10/30
# 122/122 [==============================] - 16s 134ms/step - loss: 2.8626 - accuracy: 0.4288 - val_loss: 4.5881 - val_accuracy: 0.3205
# Epoch 11/30
# 122/122 [==============================] - 19s 155ms/step - loss: 2.7610 - accuracy: 0.4331 - val_loss: 5.1245 - val_accuracy: 0.3267
# Epoch 12/30
# 122/122 [==============================] - 16s 134ms/step - loss: 2.6637 - accuracy: 0.4378 - val_loss: 4.2395 - val_accuracy: 0.2261
# Epoch 13/30
# 122/122 [==============================] - 17s 142ms/step - loss: 2.5767 - accuracy: 0.4321 - val_loss: 3.5131 - val_accuracy: 0.3130
# Epoch 14/30
# 122/122 [==============================] - 16s 131ms/step - loss: 2.4916 - accuracy: 0.4342 - val_loss: 7.7618 - val_accuracy: 0.1702
# Epoch 15/30
# 122/122 [==============================] - 16s 133ms/step - loss: 2.4260 - accuracy: 0.4429 - val_loss: 3.7722 - val_accuracy: 0.2149
# Epoch 16/30
# 122/122 [==============================] - 16s 133ms/step - loss: 2.3653 - accuracy: 0.4334 - val_loss: 5.2043 - val_accuracy: 0.3155
# Epoch 17/30
# 122/122 [==============================] - 18s 145ms/step - loss: 2.2969 - accuracy: 0.4434 - val_loss: 4.7315 - val_accuracy: 0.3217
# Epoch 18/30
# 122/122 [==============================] - 16s 133ms/step - loss: 2.2326 - accuracy: 0.4511 - val_loss: 3.5455 - val_accuracy: 0.1764
# Epoch 19/30
# 122/122 [==============================] - 16s 132ms/step - loss: 2.2102 - accuracy: 0.4449 - val_loss: 7.9723 - val_accuracy: 0.2137
# Epoch 20/30
# 122/122 [==============================] - 17s 141ms/step - loss: 2.1599 - accuracy: 0.4483 - val_loss: 5.0537 - val_accuracy: 0.4149
# Epoch 21/30
# 122/122 [==============================] - 17s 137ms/step - loss: 2.1045 - accuracy: 0.4518 - val_loss: 4.4915 - val_accuracy: 0.2323
# Epoch 22/30
# 122/122 [==============================] - 16s 134ms/step - loss: 2.0625 - accuracy: 0.4554 - val_loss: 6.7972 - val_accuracy: 0.0460
# Epoch 23/30
# 122/122 [==============================] - 17s 141ms/step - loss: 2.0292 - accuracy: 0.4477 - val_loss: 6.6514 - val_accuracy: 0.3143
# Epoch 24/30
# 122/122 [==============================] - 16s 134ms/step - loss: 1.9780 - accuracy: 0.4570 - val_loss: 2.9032 - val_accuracy: 0.2261
# Epoch 25/30
# 122/122 [==============================] - 16s 129ms/step - loss: 1.9472 - accuracy: 0.4460 - val_loss: 7.4044 - val_accuracy: 0.3143
# Epoch 26/30
# 122/122 [==============================] - 16s 131ms/step - loss: 1.9200 - accuracy: 0.4565 - val_loss: 9.5450 - val_accuracy: 0.0509
# Epoch 27/30
# 122/122 [==============================] - 17s 136ms/step - loss: 1.9004 - accuracy: 0.4580 - val_loss: 3.4936 - val_accuracy: 0.0472
# Epoch 28/30
# 122/122 [==============================] - 18s 150ms/step - loss: 1.8807 - accuracy: 0.4549 - val_loss: 5.0620 - val_accuracy: 0.0658
# Epoch 29/30
# 122/122 [==============================] - 16s 131ms/step - loss: 1.8398 - accuracy: 0.4506 - val_loss: 3.3794 - val_accuracy: 0.2137
# Epoch 30/30
# 122/122 [==============================] - 16s 133ms/step - loss: 1.8621 - accuracy: 0.4539 - val_loss: 4.0561 - val_accuracy: 0.2870
# Epoch 1/20
# 122/122 [==============================] - 49s 169ms/step - loss: 2.5873 - accuracy: 0.4080 - val_loss: 5.6910 - val_accuracy: 0.3143
# Epoch 2/20
# 122/122 [==============================] - 20s 160ms/step - loss: 2.0503 - accuracy: 0.4785 - val_loss: 7.8462 - val_accuracy: 0.3143
# Epoch 3/20
# 122/122 [==============================] - 20s 164ms/step - loss: 1.8663 - accuracy: 0.5138 - val_loss: 5.8711 - val_accuracy: 0.3143
# Epoch 4/20
# 122/122 [==============================] - 20s 163ms/step - loss: 1.7818 - accuracy: 0.5318 - val_loss: 3.2597 - val_accuracy: 0.3342
# Epoch 5/20
# 122/122 [==============================] - 19s 155ms/step - loss: 1.7099 - accuracy: 0.5346 - val_loss: 2.7576 - val_accuracy: 0.2522
# Epoch 6/20
# 122/122 [==============================] - 20s 163ms/step - loss: 1.6538 - accuracy: 0.5497 - val_loss: 2.6842 - val_accuracy: 0.1776
# Epoch 7/20
# 122/122 [==============================] - 19s 156ms/step - loss: 1.6599 - accuracy: 0.5499 - val_loss: 2.2557 - val_accuracy: 0.2882
# Epoch 8/20
# 122/122 [==============================] - 19s 157ms/step - loss: 1.5920 - accuracy: 0.5666 - val_loss: 1.7752 - val_accuracy: 0.4211
# Epoch 9/20
# 122/122 [==============================] - 21s 169ms/step - loss: 1.5775 - accuracy: 0.5725 - val_loss: 1.5623 - val_accuracy: 0.5068
# Epoch 10/20
# 122/122 [==============================] - 19s 158ms/step - loss: 1.5345 - accuracy: 0.5830 - val_loss: 1.4647 - val_accuracy: 0.5752
# Epoch 11/20
# 122/122 [==============================] - 20s 164ms/step - loss: 1.5447 - accuracy: 0.5886 - val_loss: 1.4210 - val_accuracy: 0.5876
# Epoch 12/20
# 122/122 [==============================] - 19s 156ms/step - loss: 1.5039 - accuracy: 0.6030 - val_loss: 1.3652 - val_accuracy: 0.6174
# Epoch 13/20
# 122/122 [==============================] - 21s 169ms/step - loss: 1.4665 - accuracy: 0.6091 - val_loss: 1.3497 - val_accuracy: 0.6137
# Epoch 14/20
# 122/122 [==============================] - 20s 165ms/step - loss: 1.4571 - accuracy: 0.6127 - val_loss: 1.3551 - val_accuracy: 0.6286
# Epoch 15/20
# 122/122 [==============================] - 20s 164ms/step - loss: 1.4305 - accuracy: 0.6212 - val_loss: 1.3090 - val_accuracy: 0.6584
# Epoch 16/20
# 122/122 [==============================] - 19s 158ms/step - loss: 1.4255 - accuracy: 0.6250 - val_loss: 1.3365 - val_accuracy: 0.6360
# Epoch 17/20
# 122/122 [==============================] - 20s 165ms/step - loss: 1.3856 - accuracy: 0.6342 - val_loss: 1.3487 - val_accuracy: 0.6273
# Epoch 18/20
# 122/122 [==============================] - 19s 157ms/step - loss: 1.3432 - accuracy: 0.6481 - val_loss: 1.2910 - val_accuracy: 0.6658
# Epoch 19/20
# 122/122 [==============================] - 20s 162ms/step - loss: 1.3627 - accuracy: 0.6409 - val_loss: 1.2352 - val_accuracy: 0.6770
# Epoch 20/20
# 122/122 [==============================] - 20s 160ms/step - loss: 1.3424 - accuracy: 0.6506 - val_loss: 1.2339 - val_accuracy: 0.6584
# 40/40 [==============================] - 3s 68ms/step - loss: 1.2548 - accuracy: 0.6696
# Test accuracy: 0.6695583462715149


# Etapa 2 - Número 2 - Classificador binário e SVM (1.000 imagens de cada classe):
# Binary Classifier Accuracy: 0.7148936170212766
# Binary Classifier Confusion Matrix:
# [[154  46]
#  [ 88 182]]
# Binary Classifier Classification Report:
#               precision    recall  f1-score   support

#     Negative       0.64      0.77      0.70       200
#       Others       0.80      0.67      0.73       270

#     accuracy                           0.71       470
#    macro avg       0.72      0.72      0.71       470
# weighted avg       0.73      0.71      0.72       470

# Multiclass Classifier Accuracy: 0.5468085106382978
# Multiclass Classifier Confusion Matrix:
# [[  6   4   4   1   2   0]
#  [  7  18   6  10  38   0]
#  [  0   6  33   7   6   6]
#  [  2   6   9  23  56   3]
#  [  0  12   3  10 175   0]
#  [  0   5   6   2   2   2]]
# Multiclass Classifier Classification Report:
#                                      precision    recall  f1-score   support

#                               ASC-H       0.40      0.35      0.38        17
#                              ASC-US       0.35      0.23      0.28        79
#                                HSIL       0.54      0.57      0.55        58
#                                LSIL       0.43      0.23      0.30        99
# Negative for intraepithelial lesion       0.63      0.88      0.73       200
#                                 SCC       0.18      0.12      0.14        17

#                            accuracy                           0.55       470
#                           macro avg       0.42      0.40      0.40       470
#                        weighted avg       0.51      0.55      0.51       470